In [1]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf


raw_csv_data=np.loadtxt('Audiobooks_data.csv',delimiter=',')

unscaled_inputs_all=raw_csv_data[:,1:-1]

targets_all=raw_csv_data[:,-1]




In [2]:
shuffled_indices=np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all=unscaled_inputs_all[shuffled_indices]

targets_all=targets_all[shuffled_indices]
                        



# balencing_data

In [3]:
num_one_targets=int(np.sum(targets_all))

indices_to_remove=[]

zero_target_counter=0
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_target_counter +=1
        if zero_target_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
 
targets_all_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)

# standardizing_data

In [4]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

In [5]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_scaled_inputs=scaled_inputs[shuffled_indices]

shuffled_targets=targets_all_equal_priors[shuffled_indices]

# split into train validation and tets datasets

In [6]:
sample_count=shuffled_scaled_inputs.shape[0]

train_sample_count=int(0.80*sample_count)

validation_sample_count=int(0.1*sample_count)

test_sample_count=sample_count-train_sample_count-validation_sample_count

train_input=shuffled_scaled_inputs[:train_sample_count]
train_target=shuffled_targets[:train_sample_count]

validation_input=shuffled_scaled_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_target=shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

test_input=shuffled_scaled_inputs[train_sample_count+validation_sample_count:]
test_target=shuffled_targets[train_sample_count+validation_sample_count:]

print(sample_count)
print(np.sum(train_target))
print(np.sum(validation_target))
print(np.sum(test_target))


4474
1794.0
219.0
224.0


In [7]:
train_input



array([[-0.94761239, -0.853997  ,  0.61381346, ..., -0.43427123,
        -0.1456124 ,  0.34837297],
       [-0.94761239, -0.853997  ,  0.17558209, ..., -0.43427123,
        -0.1456124 , -0.68528793],
       [ 0.13304507, -0.23782125, -0.36913539, ...,  1.88472077,
        -0.1456124 , -0.76065903],
       ...,
       [-2.56859858, -1.47017274, -0.36913539, ..., -0.43427123,
        -0.1456124 , -0.76065903],
       [-1.81213836, -1.34693759, -0.36913539, ..., -0.43427123,
        -0.1456124 ,  1.44663767],
       [ 0.13304507, -0.23782125, -0.36913539, ..., -0.43427123,
        -0.1456124 , -0.30843239]])

# save datasts into npz file

In [8]:
np.savez('Audiobook_data_train',inputs=train_input,targets=train_target)
np.savez('Audiobook_data_validation',inputs=validation_input,targets=validation_target)
np.savez('Audiobook_data_test',inputs=test_input,targets=test_target)

In [9]:
npz1=np.load('Audiobook_data_train.npz')
train_inputs=npz1['inputs'].astype(np.float)
train_targets=npz1['targets'].astype(np.int)

npz2=np.load('Audiobook_data_validation.npz')
validation_inputs=npz2['inputs'].astype(np.float)
validation_targets=npz2['targets'].astype(np.int)

npz3=np.load('Audiobook_data_test.npz')
test_inputs=npz3['inputs'].astype(np.float)
test_targets=npz3['targets'].astype(np.int)

print(len(train_inputs))
print(len(train_targets))

3579
3579


# model

In [10]:
input_size=10
output_size=2
hidden_layers_size=10

model=tf.keras.Sequential([tf.keras.layers.Dense(hidden_layers_size,activation='relu'),
                           tf.keras.layers.Dense(hidden_layers_size,activation='relu'),
                           tf.keras.layers.Dense(output_size,activation='softmax')
                          ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size=150
max_epochs=100
early_stopping=tf.keras.callbacks.EarlyStopping(patience=1)
 
model.fit(train_inputs,train_targets,batch_size=batch_size,callbacks=[early_stopping],epochs=max_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)


Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 3s - loss: 0.7390 - accuracy: 0.5136 - val_loss: 0.6860 - val_accuracy: 0.5213
Epoch 2/100
3579/3579 - 0s - loss: 0.6787 - accuracy: 0.5546 - val_loss: 0.6539 - val_accuracy: 0.5615
Epoch 3/100
3579/3579 - 0s - loss: 0.6517 - accuracy: 0.6099 - val_loss: 0.6274 - val_accuracy: 0.6421
Epoch 4/100
3579/3579 - 0s - loss: 0.6245 - accuracy: 0.6695 - val_loss: 0.5990 - val_accuracy: 0.6957
Epoch 5/100
3579/3579 - 0s - loss: 0.5966 - accuracy: 0.7016 - val_loss: 0.5707 - val_accuracy: 0.7226
Epoch 6/100
3579/3579 - 0s - loss: 0.5688 - accuracy: 0.7251 - val_loss: 0.5430 - val_accuracy: 0.7248
Epoch 7/100
3579/3579 - 0s - loss: 0.5428 - accuracy: 0.7332 - val_loss: 0.5195 - val_accuracy: 0.7293
Epoch 8/100
3579/3579 - 0s - loss: 0.5206 - accuracy: 0.7413 - val_loss: 0.4983 - val_accuracy: 0.7383
Epoch 9/100
3579/3579 - 0s - loss: 0.5012 - accuracy: 0.7471 - val_loss: 0.4815 - val_accuracy: 0.7450
Epoch 10/100
3579/3579 - 0